# Summaries of timing tests

We compare several optimizers in both [R](http://www.R-project.org) and [Julia](http://julialang.org) fitting several linear mixed models.  In __R__ the optimizers are called from `lmer` from the [lme4 package](https://github.com/lme4/lme4) (version 1.1-8). In __Julia__ the `lmm` function from the [MixedModels package](https://github.com/dmbates/MixedModels.jl) calls the optimizers.

There are differences in the model formulations in `lme4` and in `MixedModels`.  The numerical representation of the model in `lme4` and the method of evaluating the objective, described in [this paper](http://arxiv.org/abs/1406.5823), is the same for all models.  In `MixedModels` there are specialized representations for some model forms, such as models with a single grouping factor for the random effects.  Some of the specialized representations allow for evaluation of the gradient of the objects, which can enhance convergence (but, interestingly, sometimes can impede convergence).

## Methodology

To provide consistency we have copied all the data sets used in the timings to the `Timings` package itself.  We have done all timings on the same computer.  This computer has a relatively recent Intel processor and we used the [Intel Math Kernel Library (MKL)](https://software.intel.com/en-us/intel-mkl) with Julia.  We attempted to use [Revolution R Open (RRO)](www.revolutionanalytics.com/revolution-r-open) as the R implementation as it can be configured with MKL.  However, we ran into version problems with this so we used the standard Ubuntu version of R linked against OpenBLAS, which is also multi-threaded.

Variables were renamed in the pattern:
- __Y__ the response
- __A__, __B__, $\dots$ categorical covariates
- __G__, __H__, __I__, $\dots$ grouping factors for random effects
- __U__, __V__, $\dots$ (skipping __Y__) continuous covariates

The data are saved in [JSON (JavaScript Object Notation)](http://json.org) files in the directory accessible as
```r
system.file("JSON",package="Timings")
```
within R.  The directory name will end with `./Timings/inst/JSON/` in the package source directory, for example the result of cloning the [github repository](https://github.com/Stat990-033/Timings).

The `Timings` package for __R__ provides a `retime` function that takes the name of one of these JSON files and, optionally, the name of a file with the updated timings.  Similarly there are some source files for Julia retimings.


In [4]:
include("../julia/retime.jl")

retime (generic function with 2 methods)

In [5]:
retime("../JSON/Alfalfa.json","/tmp/Alfalfa.json")

dsname => "Alfalfa"
form => Formula: Y ~ 1 + B * A + (1 | G)
lmer: bobyqa -10.8102280685992, 0.50500000000001
lmer: Nelder_Mead -10.8102280684346, 0.0220000000000056
lmer: NLOPT_LN_BOBYQA -10.8102280685581, 0.0419999999999874
lmer: NLOPT_LN_COBYLA -10.8102280653537, 0.0210000000000008
lmer: NLOPT_LN_NELDERMEAD -10.8102279729183, 0.0210000000000008
lmer: NLOPT_LN_SBPLX -10.8102279729183, 0.0219999999999914
lmer: optimx:L-BFGS-B -10.810228068595, 0.108999999999995
lmer: optimx:nlminb -10.8102280685992, 0.129000000000005
lmer: optimx:spg -10.8102280685971, 0.244
lmer: optimx:bobyqa -10.8102280685992, 0.108999999999995
lmm: LN_BOBYQA -10.810228068599187, 0.001749794
lmm: LN_COBYLA -10.810228068523372, 0.009258709
lmm: LN_NELDERMEAD -10.810228012912884, 0.002031282
lmm: LN_SBPLX -10.810228068523372, 0.003245384
lmm: LD_MMA -10.81022806859886, 0.0018882
lmm: LD_CCSAQ -10.81022806859869, 0.001748693
lmm: LD_SLSQP -10.810228068599187, 0.001395835
lmm: LD_LBFGS -10.810228068599173, 0.00699064
l

5003

The names of the optimizers used with `lmm` are those from the [NLopt package](https://github.com/JuliaOpt/NLopt.jl) for __Julia__.  Names that begin with `LD_` are gradient-based methods.  Names that begin with `LN_` are derivative-free methods.  There is one other derivative-free method, `LN_PRAXIS`, available in the `NLopt` package but, for some reason, it can hang on very simple problems like this.  Frequently we just omit it.

The optimizers used with `lmer` include the `Nelder_Mead` optimizer built into the `lme4` package, the `bobyqa` optimizer from the [minqa package](http://cran.rstudio.com/web/packages/minqa/index.html), the derivative-free optimizers from the [nloptr package](http://cran.rstudio.com/web/packages/nloptr/index.html) and several optimizers from the [optimx pacakge](http://cran.rstudio.com/web/packages/optimx/index.html).

The optimizers from `nloptr` (i.e. those whose names begin with `NLOPT_LN_`) use the same underlying code as do the similarly named optimizers in the `NLopt` package for __Julia__.  The number of iterations to convergence should be similar for the same underlying code, although not nessarily exactly the same because the evaluation of the objective in __R__ and in __Julia__ may produce slightly different answers.

The `optimx:bobyqa` optimizer is just a wrapper around `bobyqa` (bounded optimization by quadratic approximation) from the `minqa` package and should behave similarly.

The numbers following the optimizer name are the value of the criterion (negative twice the log-likelihood, lower is better) achieved and the elapsed time.  The `Alfalfa` example is a particularly easy one and all of the optimizerws converge to an objective value close to -10.81023 in less than 0.6 seconds.

## Determining convergence

We chose to compare the achieved objective value to the lowest objective value for that data set/model combination. This quantity is labelled as `excess` in some tables below.